<a href="https://colab.research.google.com/github/ymuto0302/semi_2020/blob/main/classification_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (準備) Google Drive のマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls /content/drive/"My Drive"/Semi2020/LivedoorNewsCorpus/text

dokujo-tsushin	kaden-channel	movie-enter  smax	   topic-news
it-life-hack	livedoor-homme	peachy	     sports-watch


## 以下のサイトを参考に（コピペして）MeCab + Neologd をインストールする
Google ColabにMeCabとipadic-NEologdをインストールする

https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1

In [9]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


## 分類実験
LiveDoor News Corpus のデータを対象として９クラス分類問題を解く

In [13]:
#coding:utf-8

'''
livedoorニュースコーパスの分類 (9クラス問題)
https://www.rondhuit.com/download.html#ldcc
'''
import numpy as np
import MeCab
import gensim
import glob
import sys
import datetime

mecab = MeCab.Tagger("-Ochasen")

'''
文を形態素に分割する関数
'''
def tokenizer(text, acceptPOS=['名詞', '形容詞', '動詞']):
	result = list()
	node = mecab.parse(text)
	for r in node.split('\n'):
		surface = r.split('\t')[0]
		if surface == 'EOS': break

		features = r.split('\t')[3]
		if len(surface) > 1:
			result.append(surface)

	return result

'''
データセットの読み込み
'''
def read_dataset():
	docs = [] # 文書集合
	labels = [] # ラベル集合

	# livedoorニュースコーパスにて提供されているニュース記事のクラス
	class_name = ['it-life-hack', 'movie-enter', 'sports-watch',
				  'kaden-channel', 'peachy', 'topic-news',
				  'dokujo-tsushin', 'livedoor-homme', 'smax']

	print("=== read dataset ===", file=sys.stderr)

	for cn in class_name:
		print("*** reading {} ***".format(cn), file=sys.stderr)
	
		paths = glob.glob(r'/content/drive/My Drive/Semi2020/LivedoorNewsCorpus/text/{}/*.txt'.format(cn))
		for path in paths:
			with open(path, 'r') as f:
				tokens = []
				# 各ファイルの冒頭３行はヘッダゆえ，それらを読み飛ばす
				for line in f.readlines()[3:]:
					sentence = line.rstrip('\n')
					tokens.extend([token for token in tokenizer(sentence)])
				docs.append(tokens)
				labels.append(class_name.index(cn))

	print("Number of documents:", len(docs), file=sys.stderr)

	return docs, np.array(labels)

'''
特徴抽出
 gensim を用いた形態素のマッピングと前処理
 gensim を用いた低頻度語と高頻度語の除去
 gensim を用いた特徴抽出
'''
def feature_extraction(docs):
	# 単語と単語 ID のマッピング
	print("=== mapping ID with word ===", file=sys.stderr)
	dictionary = gensim.corpora.Dictionary(docs)
	print("original voc. size:",len(dictionary.keys()), file=sys.stderr)

	print("--- example of word mapping ---", file=sys.stderr)
	counter = 0
	for k in dictionary.token2id.keys():
		print(k, dictionary.token2id[k], file=sys.stderr)
		counter += 1
		if counter == 20: break
	
	# 低頻度語と高頻度語の除去
	# 出現回数が 10回未満，または単語が 30% の文書に登場したとき，その他を除外
	dictionary.filter_extremes(no_below=10, no_above=0.3)

	print("reduced voc. size:", len(dictionary.keys()), file=sys.stderr)

	# TF-IDF 特徴量の抽出
	print("=== feature extraction ===", file=sys.stderr)
	corpus = [dictionary.doc2bow(doc) for doc in docs] # TFを記録したコーパス
	# print corpus

	tfidfModel = gensim.models.TfidfModel(corpus)
	corpus_tfidf = tfidfModel[corpus] #TF-IDFを記録したコーパス

	# 文書-単語行列：features には TF-IDF値が格納されている
	features = gensim.matutils.corpus2dense(corpus_tfidf, num_terms=len(dictionary)).T

	return features

'''
分類
'''
def classification(features, labels):
	print("=== training and testing the classifier ===", file=sys.stderr)

	from sklearn.model_selection import train_test_split
	from sklearn.neural_network import MLPClassifier

	# データセットを学習データとテストデータに分割
	x_train, x_test, y_train, y_test = train_test_split(features, labels, train_size=0.8, test_size=0.2)

	# 識別器(multi-layer perceptron)の定義
	# classifier = MLPClassifier(hidden_layer_sizes=(200, 100, 50, 10), verbose=True)
	classifier = MLPClassifier(hidden_layer_sizes=(200, 10), verbose=True)
	
	# from sklearn.ensemble import RandomForestClassifier
	# classifier = RandomForestClassifier(n_estimators=100)

	# 学習
	classifier.fit(x_train, y_train)

	# 予測 (分類)
	print("accuracy:", classifier.score(x_test, y_test))

# データセットの読み込み：docs は文書集合，labels はラベル集合
docs, labels = read_dataset()

'''
docs の中身のイメージ
[['今朝', '雪', '積もっ', 'い'],
	['雪','いえ','アナ', '雪', '女王'],
	['雪', 'いえ', '白雪姫'],
	['明日', '雪', '降る']
]
'''

# 特徴抽出
features = feature_extraction(docs)
print("shape of features", features.shape, file=sys.stderr)
print("shape of labels", labels.shape, file=sys.stderr)

# 分類実験
classification(features, labels)


[]
[]
[]
[]
[]
[]
[]
[]
[]


Number of documents: 0
=== mapping ID with word ===
original voc. size: 0
--- example of word mapping ---
reduced voc. size: 0
=== feature extraction ===
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:502: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)


ValueError: ignored